# Understanding retries
Here description of what this does.

In [1]:
import datetime
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import scan
import re
import statistics

import numpy as np
import pandas as pd

es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)
jobs = []

### select period

In [2]:
start_date = datetime.datetime(2018, 2, 1, 0, 0, 0)
#start_date = datetime.datetime(2018, 1, 30, 0, 0, 0)
curre_date = datetime.datetime(2018, 3, 5, 23, 59, 59)
#curre_date = datetime.datetime.utcnow()
days_of_data = (curre_date - start_date).days
timemargin = datetime.timedelta(1)

### selecting indices to scan

In [3]:
# get job archive indices
indices = es.cat.indices(index="jobs_archive_20*", h="index", request_timeout=600).split('\n')
indices = sorted(indices)
indices = [x for x in indices if x != '']
#for i in indices:
#    print(i)

selected_indices=[] 
for d in range(days_of_data+1):
    td = start_date + datetime.timedelta(d)
    ind = 'jobs_archive_'+td.strftime('%Y-%m-%d')
    if ind in indices:
        selected_indices.append(ind)

job_indices = ','.join(selected_indices)
print(job_indices)

jobs_archive_2018-02-01,jobs_archive_2018-02-02,jobs_archive_2018-02-03,jobs_archive_2018-02-04,jobs_archive_2018-02-05,jobs_archive_2018-02-06,jobs_archive_2018-02-07,jobs_archive_2018-02-08,jobs_archive_2018-02-09,jobs_archive_2018-02-10,jobs_archive_2018-02-11,jobs_archive_2018-02-12,jobs_archive_2018-02-13,jobs_archive_2018-02-14,jobs_archive_2018-02-15,jobs_archive_2018-02-16,jobs_archive_2018-02-17,jobs_archive_2018-02-18,jobs_archive_2018-02-19,jobs_archive_2018-02-20,jobs_archive_2018-02-21,jobs_archive_2018-02-22,jobs_archive_2018-02-23,jobs_archive_2018-02-24,jobs_archive_2018-02-25,jobs_archive_2018-02-26,jobs_archive_2018-02-27,jobs_archive_2018-02-28,jobs_archive_2018-03-01,jobs_archive_2018-03-02,jobs_archive_2018-03-03,jobs_archive_2018-03-04,jobs_archive_2018-03-05


### running the scan

In [4]:
job_query = {
    "size": 0,
    "_source": [ "proddblock" ],
    'query':{
#        "match_all": {}
             'bool':{
                    'must':[
                         { "term": {"jobstatus": "finished" } },
#                         { "regexp": {"creationhost" : "hammercloud-ai*" } },
#                         { "term": {"nevents" : "25" } },
                     ],
#                    'should':[
                       # { "term": {"prodsourcelabel": "rc_test" } },
                       # { "term": {"prodsourcelabel": "prod_test" } },
                       # { "term": {"prodsourcelabel": "managed" } }
#                        { "term": {"jobstatus": "finished" } },
#                        { "term": {"jobstatus": "failed" } },
#                    ]
             }
    }
                
}

scroll = scan(client=es, index=job_indices, query=job_query, scroll='1000000m', timeout="1000000m", size=10000)

count = 0
for res in scroll:
    count += 1
    
    #print(res)
    #if count>10000: break
    if count % 100000 == 0:
        print(count)
    
    r = res['_source']
    proddblock = r['proddblock']
    
    if proddblock is not None:
        if 'data17' in proddblock and 'DAOD_RPVLL' in proddblock:
            print(proddblock)

print ('end')

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000
1